In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("churn.csv")

In [3]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [4]:
df.isnull().sum().sum()

0

In [5]:
df=df.drop("customerID",axis=1)
df = pd.get_dummies(df)
df = df.drop("Churn_No",axis=1)

In [6]:
df

,SeniorCitizen,tenure,MonthlyCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,TotalCharges_996.45,TotalCharges_996.85,TotalCharges_996.95,TotalCharges_997.65,TotalCharges_997.75,TotalCharges_998.1,TotalCharges_999.45,TotalCharges_999.8,TotalCharges_999.9,Churn_Yes
0,0,1,29.85,True,False,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
1,0,34,56.95,False,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,2,53.85,False,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,0,45,42.30,False,True,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
4,0,2,70.70,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,24,84.80,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
7039,0,72,103.20,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
7040,0,11,29.60,True,False,False,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
7041,1,4,74.40,False,True,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,True


In [7]:
for c in df:
    if df[c].dtype=='bool':
        df[c] = df[c].map({True: 1, False: 0})
X = df.drop("Churn_Yes", axis=1)
y = df["Churn_Yes"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Columns: 6576 entries, SeniorCitizen to Churn_Yes
dtypes: float64(1), int64(6575)
memory usage: 353.4 MB


In [9]:
'''y_train = y_train.astype("category")
for c in X_train:
    X_train[c] = X_train[c].astype("category")
for c in X_test:
    X_test[c] = X_test[c].astype("category")'''

'y_train = y_train.astype("category")\nfor c in X_train:\n    X_train[c] = X_train[c].astype("category")\nfor c in X_test:\n    X_test[c] = X_test[c].astype("category")'

In [10]:
X_train, y_train

(      SeniorCitizen  tenure  MonthlyCharges  gender_Female  gender_Male  \
 3522              0      37           19.80              0            1   
 2470              0      67           25.25              1            0   
 3976              0       4           24.45              1            0   
 5390              1      12           99.45              0            1   
 2559              0      72          115.55              0            1   
 ...             ...     ...             ...            ...          ...   
 3537              0      25           53.65              0            1   
 2500              0      50           75.50              0            1   
 4000              1      39           99.75              1            0   
 4756              0      31           72.45              0            1   
 3884              0      18           88.85              1            0   
 
       Partner_No  Partner_Yes  Dependents_No  Dependents_Yes  PhoneService_No  \
 352

In [11]:
'''smt = SMOTE()
X_train, y_train = smt.fit_resample(X_train, y_train)'''

'smt = SMOTE()\nX_train, y_train = smt.fit_resample(X_train, y_train)'

In [12]:
#clf = xgb.XGBClassifier(objective="binary:logistic",eta=0.1,tree_method="approx")
maxy=[]
maxr=0;
for i in range(0,100,1):
    params = {"objective":"binary:logistic",'learning_rate': 0.2, 'seed':123, 'max_depth': 12, 'alpha':10, 'colsample_bytree': i/100}
    clf = xgb.XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    a = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    y_pred2 = clf.predict(X_train)
    accuracy2 = accuracy_score(y_train, y_pred2)
    precision2 = precision_score(y_train, y_pred2)
    recall2 = recall_score(y_train, y_pred2)
    f12 = f1_score(y_train, y_pred2)
    if (accuracy>maxr):
        maxr=accuracy
        maxy=[i,accuracy,accuracy2]
print(maxy)

C:\Users\subha\anaconda3\envs\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\subha\anaconda3\envs\gpu_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[70, 0.8239886444286728, 0.8239261625843095]


In [13]:
print(maxy)

[70, 0.8239886444286728, 0.8239261625843095]


In [30]:
params = {"objective":"binary:logistic",'learning_rate': 0.2, 'seed':123, 'max_depth': 12, 'alpha':10, 'colsample_bytree': 0.7}
clf = xgb.XGBClassifier(**params)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [31]:
confusion_matrix(y_test, y_pred)

array([[949,  87],
       [161, 212]], dtype=int64)

In [32]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [33]:
accuracy, precision, recall, f1

(0.8239886444286728,
 0.7090301003344481,
 0.5683646112600537,
 0.6309523809523809)

In [18]:
y_pred2 = clf.predict(X_train)
accuracy = accuracy_score(y_train, y_pred2)
precision = precision_score(y_train, y_pred2)
recall = recall_score(y_train, y_pred2)
f1 = f1_score(y_train, y_pred2)

In [19]:
accuracy, precision, recall, f1

(0.8281860134895279, 0.7222222222222222, 0.5735294117647058, 0.639344262295082)

In [20]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [21]:
params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

xgb_cv = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [22]:
xgb_cv

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.804292,0.001258,0.792669,0.020760
1,0.819792,0.003210,0.809427,0.014843
2,0.828684,0.004084,0.818982,0.012384
3,0.837658,0.003004,0.827273,0.010029
4,0.836981,0.003064,0.824990,0.009684
5,0.842048,0.003217,0.830079,0.008781
6,0.841668,0.003801,0.830203,0.009539
7,0.842744,0.003728,0.831732,0.009104
8,0.843702,0.003671,0.832097,0.008503
9,0.846041,0.003243,0.834039,0.009106


In [23]:
xgb_cv = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=100, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [24]:
xgb_cv

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.804292,0.001258,0.792669,0.020760
1,0.819792,0.003210,0.809427,0.014843
2,0.828684,0.004084,0.818982,0.012384
3,0.837658,0.003004,0.827273,0.010029
4,0.836981,0.003064,0.824990,0.009684
...,...,...,...,...
61,0.860002,0.003986,0.843091,0.007321
62,0.859989,0.003988,0.843081,0.007310
63,0.860035,0.004025,0.843074,0.007369
64,0.860167,0.003976,0.843162,0.007269


In [25]:
print(xgb_cv)

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.804292       0.001258       0.792669      0.020760
1         0.819792       0.003210       0.809427      0.014843
2         0.828684       0.004084       0.818982      0.012384
3         0.837658       0.003004       0.827273      0.010029
4         0.836981       0.003064       0.824990      0.009684
..             ...            ...            ...           ...
61        0.860002       0.003986       0.843091      0.007321
62        0.859989       0.003988       0.843081      0.007310
63        0.860035       0.004025       0.843074      0.007369
64        0.860167       0.003976       0.843162      0.007269
65        0.860225       0.003985       0.843197      0.007309

[66 rows x 4 columns]


In [26]:
params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 10, 'alpha': 10}
xgb_cv = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=100, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [27]:
xgb_cv

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.805471,0.001161,0.793275,0.020710
1,0.820600,0.002897,0.809543,0.014659
2,0.828986,0.003915,0.819234,0.012240
3,0.838255,0.002603,0.827520,0.010027
4,0.837555,0.002848,0.825061,0.009729
...,...,...,...,...
61,0.861704,0.003829,0.842995,0.007248
62,0.861699,0.003819,0.842989,0.007241
63,0.861721,0.003839,0.843021,0.007275
64,0.861887,0.003803,0.843125,0.007303
